In [1]:
import os
import json

def calculate_intrinsic_value(
    initial_fcf: float,
    growth_rate: float,
    terminal_multiple: float,
    discount_rate: float,
    years: int,
    cash: float,
    debt: float,
    shares: float
) -> float:
    """
    Calculate the intrinsic value per share of a stock using the DCF method.

    Parameters:
    - initial_fcf: The current free cash flow (year 0) (e.g., 60e9 for 60 billion).
    - growth_rate: The annual growth rate for FCF (e.g., 0.08 for 8%).
    - terminal_multiple: The multiple used to value the final year's FCF (e.g., 15).
    - discount_rate: The required annual return (e.g., 0.13 for 13%).
    - years: The number of years to project (e.g., 10).
    - cash: The total cash and cash equivalents.
    - debt: The total debt.
    - shares: The number of shares outstanding.

    Returns:
    - The intrinsic value per share.
    """
    total_value = 0.0

    if growth_rate > 0:
        growth_rate = growth_rate / 100  # Convert percentage to decimal.

    # Calculate the present value of projected FCF for each year.
    for year in range(1, years + 1):
        # Projected FCF for the given year (assuming compound growth)
        fcf_year = initial_fcf * (1 + growth_rate) ** year
        # Discount the FCF back to today
        present_value = fcf_year / ((1 + discount_rate) ** year)
        total_value += present_value

    # Calculate the terminal value at the end of the projection period.
    fcf_final_year = initial_fcf * (1 + growth_rate) ** years
    terminal_value = fcf_final_year * terminal_multiple
    terminal_value_pv = terminal_value / ((1 + discount_rate) ** years)
    total_value += terminal_value_pv

    # Adjust for net cash (cash minus debt) to convert from enterprise value to equity value.
    equity_value = total_value + cash - debt

    # Calculate the intrinsic value per share.
    intrinsic_value_per_share = equity_value / shares
    return intrinsic_value_per_share


def intrinsic_values_over_discount_rates(
    initial_fcf: float,
    growth_rate: float,
    terminal_multiple: float,
    years: int,
    cash: float,
    debt: float,
    shares: float,
    start_rate: float = 0.06,
    end_rate: float = 0.18,
    step: float = 0.01
) -> dict:
    """
    Calculate the intrinsic value per share for a range of discount rates.

    Parameters:
    - initial_fcf, growth_rate, terminal_multiple, years, cash, debt, shares:
        See the parameters in calculate_intrinsic_value.
    - start_rate: Starting discount rate (default 6% or 0.06).
    - end_rate: Ending discount rate (default 18% or 0.18).
    - step: Step for discount rate increments (default 0.01, i.e., 1%).

    Returns:
    - A dictionary where keys are discount rates (as decimals) and values are the intrinsic value per share.
    """
    results = {}
    discount_rate = start_rate
    while discount_rate <= end_rate + 1e-6:  # Adding tolerance for floating-point comparisons.
        iv = calculate_intrinsic_value(
            initial_fcf,
            growth_rate,
            terminal_multiple,
            discount_rate,
            years,
            cash,
            debt,
            shares
        )
        results[discount_rate] = iv
        discount_rate += step
    return results

In [2]:
ticker = 'GOOGL'            # Ticker
initial_fcf = 60e9          # current
growth_rate = 8             # FCF growth (in %)
terminal_multiple = 15      # Terminal multiple
years = 10                  # Projection period
cash = 110e9                # Total cash
debt = 27e9                 # Total debt
shares = 13e9               # Number of shares outstanding

In [3]:
# Calculate intrinsic values over discount rates from 6% to 18%
intrinsic_values = intrinsic_values_over_discount_rates(
    initial_fcf, growth_rate, terminal_multiple,
    years, cash, debt, shares,
    start_rate=0.06, end_rate=0.18, step=0.01
)

# Print the results to the console
for rate in sorted(intrinsic_values.keys()):
    print(f"{rate*100:.0f}%  ->  ${intrinsic_values[rate]:,.2f}")

# Prepare the results for JSON output.
# We convert the discount rate keys to percentage strings for readability.
output_data = {f"{rate*100:.0f}%": intrinsic_values[rate] for rate in intrinsic_values}

# Ensure the 'dcf' directory exists
output_directory = "dcf"
os.makedirs(output_directory, exist_ok=True)

# Create the output file path using the ticker.
output_file = os.path.join(output_directory, f"{ticker}.json")

# Write the results to the JSON file.
with open(output_file, 'w') as json_file:
    json.dump(output_data, json_file, indent=4)

6%  ->  $141.07
7%  ->  $130.96
8%  ->  $121.77
9%  ->  $113.41
10%  ->  $105.79
11%  ->  $98.84
12%  ->  $92.50
13%  ->  $86.70
14%  ->  $81.40
15%  ->  $76.54
16%  ->  $72.08
17%  ->  $67.99
18%  ->  $64.23
